#### Ingestion del archivo production_company

In [0]:
dbutils.widgets.text("p_enviroment","")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

##### Paso 1 - Leer el archivo Json usando "DataFrameReader" en Spark

In [0]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

In [0]:
leanguage_role_schema = StructType(fields = [
  StructField("companyId", IntegerType(), False),
  StructField("companyName", StringType(),False)
])

In [0]:
production_company_df = spark.read.schema(leanguage_role_schema).csv(f"{bronze_folder_path}/{v_file_date}/production_company")

#### Paso 2 - Agregar los campos faltnates y cambiar de nombre
##### 1. Cambiar el nombre de "companyId" a "company_id"
##### 2. Cambiar el nombre de "companyName" a "companyName"
##### 2. Agregar "enviroment" y Agregar "ingestion_date"

In [0]:
from pyspark.sql.functions import lit, current_timestamp

In [0]:
production_company_final = add_ingestion_date(production_company_df).withColumnRenamed("companyId","company_id") \
                                .withColumnRenamed("companyName","company_name") \
                                .withColumn("enviromen",lit(v_enviroment)) \
                                .withColumn("file_date",lit(v_file_date))

#### Paso 3 - Agregar el DF en formato parQuet

In [0]:
#overwrite_partition(production_company_final,"movie_silver","prodcution_company","file_date")

merge_condition = 'tgt.company_id = src.company_id AND tgt.file_date = src.file_date'
merge_delta_lake(production_company_final, "movie_silver", "production_company", silver_foler_path, merge_condition, "file_date")

In [0]:
%sql
SELECT count(1), file_date
FROM movie_silver.production_company
GROUP BY file_date

In [0]:
dbutils.notebook.exit("Success!")